In [59]:
import pandas as pd
import numpy as np
import collections
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from matplotlib.patches import ArrowStyle
from scipy import stats
from sklearn import metrics
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, log_loss
from sklearn.metrics import confusion_matrix
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
%matplotlib inline

In [2]:
df = pd.read_csv('./dat', encoding='utf-8', sep=',') # data read
# df = pd.read_csv('./dat_test', encoding='utf-8', sep=',') # data read

In [3]:
df_proc = df.copy()

In [4]:
df_proc['x0_age'] = df_proc['x0_age'].fillna(df_proc['x0_age'].median())

In [5]:
df_proc['x10'] = df_proc['x10'].fillna(df_proc['x10'].median())

In [6]:
df_hist = df_proc.copy()

In [7]:
df_proc['x58'] = df_proc['x58'].fillna(0)
df_proc['x59'] = df_proc['x59'].fillna(0)
df_proc['x60'] = df_proc['x60'].fillna(0)
df_proc['x61'] = df_proc['x61'].fillna(0)
df_proc['x62'] = df_proc['x62'].fillna(-1)
df_proc['x63'] = df_proc['x63'].fillna(0)
df_proc['x64'] = df_proc['x64'].fillna(99999999)
df_proc['x65'] = df_proc['x65'].fillna(0)
df_proc['x66'] = df_proc['x66'].fillna(0)
df_proc['x67'] = df_proc['x67'].fillna(99)
df_proc['x68'] = df_proc['x68'].fillna(0)
df_proc['x69'] = df_proc['x69'].fillna(0)
df_proc['x70'] = df_proc['x70'].fillna(0)
df_proc['x71'] = df_proc['x71'].fillna(-1)

In [8]:
# one hot encoding
df_proc = pd.get_dummies(df_proc, columns=['x9'])

In [9]:
dfs = df_proc.copy()

In [10]:
drop_col = ['x14','x18','x19','x21','x22','x23','x27','x28','x29','x30','x31','x33','x34','x35','x36','x37','x38','x40','x49','x72']
df_spray = df_proc.drop(drop_col, axis=1)

In [11]:
# 相関があるが、同数の項目値が0⇆99と異なる。「x6」は99→0へ変換する。
df_spray['x6'] = df_spray['x6'].replace(99, 0)

In [12]:
# 相関があるが、同数の項目値が0⇆99と異なる。「x6」は99→0へ変換する。
df_spray['x67'] = df_spray['x67'].replace(99, 0)

In [13]:
# 相関があるが、同数の項目値が0⇆99999999へ変換すると異なる。「x63」は0→99999999へ変換する。
df_spray['x63'] = df_spray['x63'].replace(0, 99999999)

In [14]:
df_del = df_spray.copy()

In [15]:
df_del = df_del.drop(columns='x61')
df_del = df_del.drop(columns='x65')
df_del = df_del.drop(columns='x66')
df_del = df_del.drop(columns='x70')

In [16]:
df_label = df_del.copy()

In [17]:
# ランダム・フォレストによる相互作用評価
X = df_label.drop(columns='y')
y = df_label['y']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=0)

classifier_model = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifier_model.fit(train_X, train_y)

pred_y = classifier_model.predict(test_X)
cm = confusion_matrix(test_y, pred_y)
print(cm)
print(accuracy_score(test_y, pred_y))

[[14929    61]
 [  628   132]]
0.9562539682539682


In [18]:
# Importtance
# 組み込み法（モデルベース特徴量選択：モデルが学習するタイミングで特徴量を評価する）
importance = classifier_model.feature_importances_
importances = pd.DataFrame(importance, index = X.columns, columns=['importance'])
pd.set_option('display.max_rows', 80)
importances.sort_values(by='importance', ascending=False)

,importance
x1_term,0.143358
x10,0.098021
x0_age,0.095925
x5,0.075107
x3,0.058060
x57,0.049504
x6,0.039396
x4,0.038059
x56,0.037499
x2_sex,0.025759


In [19]:
# Filter Method：
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
print(train_X.shape) # 全項目

X_train = train_X.copy()
sel.fit(X_train) # 分散0の項目を排除
X_train = X_train.loc[:, sel.get_support()]
print(X_train.shape) # 非該当項目

X_train_diff = train_X.drop(columns=X_train)

(36750, 57)
(36750, 52)


In [20]:
# 下記も削除対象とする
cols =['x46', 'x48', 'x50', 'x51'] # 'x47'残す
df_zero = df_label.copy()
df_zero = df_zero.drop(columns=cols)

In [21]:
# ②他の特徴量と値が完全に一致している変数の片方を削除
train_X_T = train_X.T
duplicated_features = train_X_T[train_X_T.duplicated()].index.values
print(duplicated_features)
# 'x47' 'x48' 'x50' 'x51' ←1つ残す、ただし全て相互作用評価結果(Importance=0)

['x47' 'x48' 'x50' 'x51']


In [22]:
df_add = df_zero.copy()
df_add['x73'] = (df_add['x0_age'] + df_add['x10'])/2
df_add['x74'] = (df_add['x5'] + df_add['x6'])/2
df_add['x75'] = (df_add['x59'] + df_add['x67'])/2
df_add['x76'] = (df_add['x59'] + df_add['x68'])/2
df_add['x77'] = (df_add['x62'] + df_add['x71'])/2
df_add['x78'] = (df_add['x63'] + df_add['x64'])/2

In [23]:
# ランダム・フォレストによる相互作用評価
X_re = df_add.drop(columns='y')
y_re = df_add['y']

train_X_re, test_X_re, train_y_re, test_y_re = train_test_split(X_re, y_re, test_size=0.3, random_state=0)

classifier_model_re = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifier_model_re.fit(train_X_re, train_y_re)

pred_y_re = classifier_model_re.predict(test_X_re)
cm_re = confusion_matrix(test_y_re, pred_y_re)
print(cm_re)
print(accuracy_score(test_y, pred_y))

[[14921    69]
 [  631   129]]
0.9562539682539682


In [24]:
# Importtance
importance_re = classifier_model_re.feature_importances_
importances_re = pd.DataFrame(importance_re, index = X_re.columns, columns=['importance'])
pd.set_option('display.max_rows', 50)
importances_re.sort_values(by='importance', ascending=False)

,importance
x1_term,0.118974
x73,0.080239
x10,0.077516
x0_age,0.073920
x5,0.059724
...,...
x62,0.000701
x71,0.000691
x67,0.000586
x68,0.000526


In [25]:
# Importanceからも削除
print(type(importances_re))
print(importances_re.shape)
importances_re = importances_re.drop('x75', axis=0)
importances_re = importances_re.drop('x76', axis=0)
importances_re = importances_re.drop('x77', axis=0)
importances_re = importances_re.drop('x78', axis=0)

<class 'pandas.core.frame.DataFrame'>
(59, 1)


In [26]:
df_select_del = df_add.copy()
del_col = ['x7', 'x8', 'x11', 'x15', 'x16', 'x24', 'x25', 'x26', 'x32', 'x39','x41', 'x44', 'x47', 'x54', 'x58', 'x59', 'x60', 'x62', 'x63', 'x64','x67', 'x68', 'x69', 'x71']
df_select_del = df_select_del.drop(columns=del_col)

In [27]:
df_light = df_select_del.copy()

In [28]:
# validation
df_X = df_light.drop(columns='y')
df_y = df_light['y']

In [29]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(df_X, df_y, test_size=0.2, random_state=0)
X_train_val_t, X_val_t, y_train_val_t, y_val_t = train_test_split(X_train_t, y_train_t, test_size=0.2, random_state=0)

In [142]:
# デフォルト
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'num_leaves': 31,
#     'num_iterations': 100,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'max_depth': ,
#     'min_data_in_leaf': 20,
# }

In [143]:
#     'boosting_type': 'gbdt', # アルゴリズム：勾配ブースティング
#     'objective': 'binary', # 目的 : 2値分類 
#     'metric': {'binary_error'}, # 評価指標 : 誤り率, binary_error（正答率）, binary_logloss(クロスエントロピー), auc
#     'num_leaves': 200, # Treeの複雑さ：（デフォルトは31）
#     'num_iterations':200, # 木の数：（デフォルトは100）
#     'learning_rate': 0.01, # 学習率：（デフォルト0.1の半分）
#     'feature_fraction': 0.9, # 各木を作成するときに使用可能な特徴量の割合：90%（デフォルト1.0）
#     'bagging_fraction': 0.8, # 使用するデータの割合：80%（デフォルト1.0）
#     'bagging_freq': 10, # 何回に一回baggingするか
#     'max_depth':10, # デフォルト：None
#     'colsample_bytree':1,
#     'n_estimators':50000, # ランダムサーチ(パラメータ範囲指定)
#     'min_data_in_leaf': 20, # 決定木のノード（葉）の最小データ数
#     'vervose': 0 # 途中経過出力：する

In [144]:
# # 自動チューニング
# import optuna.integration.lightgbm as lgb_tune

# df_train, df_val =train_test_split(df_light, test_size=0.2)
 
# col = "y"
# train_y = df_train[col]
# train_x = df_train.drop(col, axis=1)
 
# val_y = df_val[col]
# val_x = df_val.drop(col, axis=1)
 
# trains = lgb.Dataset(train_x, train_y)
# valids = lgb.Dataset(val_x, val_y)
 
# model_tune = lgb_tune.train(
#     params, trains, valid_sets=valids, 
#     verbose_eval=100, 
#     early_stopping_rounds=100,
# )
# print(model_tune.params)

# # 予測
# preds = model_tune.predict(train_x)
# # AUC
# fpr, tpr, thresholds = metrics.roc_curve(train_y, preds)
# auc = metrics.auc(fpr, tpr)
# auc

In [219]:
# ホールドアウト法→交差検証 (クロスバリデーション)に切り替える
kf = KFold(n_splits=5, shuffle=True, random_state=0)
predicted_df = pd.DataFrame({'index':0, 'pred':0}, index=[1])

val_indexs=[]
i=0
for train_index, val_index in kf.split(df_light.index):
    X_train = df_light.drop('y', axis=1).iloc[train_index]
    y_train = df_light['y'].iloc[train_index]
    X_test = df_light.drop('y', axis=1).iloc[val_index]
    y_test = df_light['y'].iloc[val_index]
    
# XGBoost Modeling
    clf = XGBClassifier(
            objective='binary:logistic',
            num_leaves=31,
            num_iterations=100,
            learning_rate=0.1,
            feature_fraction=1,
            subsample=1,
            max_depth=1,
            min_data_in_leaf=5
    )
    metLearn=CalibratedClassifierCV(clf)
    clf.fit(X_train, y_train)

[17:10:15] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { feature_fraction } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:10:15] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:16] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { feature_fraction } might no

In [242]:
# 予測
Predictions = clf.predict_proba(X_test)

# AUC
fpr, tpr, thresholds = metrics.roc_curve(y_test, Predictions[:,1])
auc = metrics.auc(fpr, tpr)
auc

0.8758822973633897

In [243]:
y_pred = np.where(Predictions[:, 1] < 0.5, 0, 1) # 閾値=0.5

In [244]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(log_loss(y_test, y_pred))

0.9541904761904761
0.08531746031746032
0.6825396825396826
0.15167548500881833
1.5822064226572048


In [168]:
# KS値
# K-S検定（コルモゴロフ-スミルノフ検定）
clf= RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
        max_depth=25, max_features='auto', max_leaf_nodes=None,
        min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=1, min_samples_split=15,
        min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
        oob_score=False, random_state=0, verbose=0, warm_start=False)

clf.fit(X_train, y_train)
pre=clf.predict_proba(X_train)
pre_pd = pd.DataFrame(pre[:,0])
pre_pd['y'] = y_train

In [224]:
# 変数のAUC
X_train_C = X_train.loc[:,['x0_age','x1_term','x9_C']].copy()
X_train_D = X_train.loc[:,['x0_age','x1_term','x9_D']].copy()
X_train_H = X_train.loc[:,['x0_age','x1_term','x9_H']].copy()
X_train_K = X_train.loc[:,['x0_age','x1_term','x9_K']].copy()
X_train_N = X_train.loc[:,['x0_age','x1_term','x9_N']].copy()
X_train_Q = X_train.loc[:,['x0_age','x1_term','x9_Q']].copy()
X_train_R = X_train.loc[:,['x0_age','x1_term','x9_R']].copy()
X_train_S = X_train.loc[:,['x0_age','x1_term','x9_S']].copy()
X_train_T = X_train.loc[:,['x0_age','x1_term','x9_T']].copy()

X_test_C = X_test.loc[:,['x0_age','x1_term','x9_C']].copy()
X_test_D = X_test.loc[:,['x0_age','x1_term','x9_D']].copy()
X_test_H = X_test.loc[:,['x0_age','x1_term','x9_H']].copy()
X_test_K = X_test.loc[:,['x0_age','x1_term','x9_K']].copy()
X_test_N = X_test.loc[:,['x0_age','x1_term','x9_N']].copy()
X_test_Q = X_test.loc[:,['x0_age','x1_term','x9_Q']].copy()
X_test_R = X_test.loc[:,['x0_age','x1_term','x9_R']].copy()
X_test_S = X_test.loc[:,['x0_age','x1_term','x9_S']].copy()
X_test_T = X_test.loc[:,['x0_age','x1_term','x9_T']].copy()

In [235]:
# ロジスティック回帰モデル
lr = LogisticRegression()

In [236]:
lr.fit(X_train_C, y_train)
y_pred_C = lr.predict_proba(X_test_C)

lr.fit(X_train_D, y_train)
y_pred_D = lr.predict_proba(X_test_D)

lr.fit(X_train_H, y_train)
y_pred_H = lr.predict_proba(X_test_H)

lr.fit(X_train_K, y_train)
y_pred_K = lr.predict_proba(X_test_K)

lr.fit(X_train_N, y_train)
y_pred_N = lr.predict_proba(X_test_N)

lr.fit(X_train_Q, y_train)
y_pred_Q = lr.predict_proba(X_test_Q)

lr.fit(X_train_R, y_train)
y_pred_R = lr.predict_proba(X_test_R)

lr.fit(X_train_S, y_train)
y_pred_S = lr.predict_proba(X_test_S)

lr.fit(X_train_T, y_train)
y_pred_T = lr.predict_proba(X_test_T)

In [239]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_C[:,1])
auc_C = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_D[:,1])
auc_D = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_H[:,1])
auc_H = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_K[:,1])
auc_K = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_N[:,1])
auc_N = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_Q[:,1])
auc_Q = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_R[:,1])
auc_R = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_S[:,1])
auc_S = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_T[:,1])
auc_T = metrics.auc(fpr, tpr)

In [240]:
print(auc_C)
print(auc_D)
print(auc_H)
print(auc_K)
print(auc_N)
print(auc_Q)
print(auc_R)
print(auc_S)
print(auc_T)

0.2656136859505707
0.7349830805338008
0.733441094691845
0.7320518485171847
0.7344143014348596
0.7350860979312359
0.734763945260644
0.7346024719411575
0.7273721988795518


In [65]:
# 合成変数
X_train_3 = X_train.loc[:,['x0_age','x1_term','x73']].copy()
X_train_4 = X_train.loc[:,['x0_age','x1_term','x74']].copy()
X_train_5 = X_train.loc[:,['x0_age','x1_term','x75']].copy()
X_train_6 = X_train.loc[:,['x0_age','x1_term','x76']].copy()
X_train_7 = X_train.loc[:,['x0_age','x1_term','x77']].copy()
X_train_8 = X_train.loc[:,['x0_age','x1_term','x78']].copy()

In [66]:
X_test_3 = X_test.loc[:,['x0_age','x1_term','x73']].copy()
X_test_4 = X_test.loc[:,['x0_age','x1_term','x74']].copy()
X_test_5 = X_test.loc[:,['x0_age','x1_term','x75']].copy()
X_test_6 = X_test.loc[:,['x0_age','x1_term','x76']].copy()
X_test_7 = X_test.loc[:,['x0_age','x1_term','x77']].copy()
X_test_8 = X_test.loc[:,['x0_age','x1_term','x78']].copy()

In [67]:
# ロジスティック回帰モデル
lr = LogisticRegression()

In [68]:
lr.fit(X_train_3, y_train)
y_pred_3 = lr.predict(X_train_3)

lr.fit(X_train_4, y_train)
y_pred_4 = lr.predict(X_train_4)

lr.fit(X_train_5, y_train)
y_pred_5 = lr.predict(X_train_5)

lr.fit(X_train_6, y_train)
y_pred_6 = lr.predict(X_train_6)

lr.fit(X_train_7, y_train)
y_pred_7 = lr.predict(X_train_7)

lr.fit(X_train_8, y_train)
y_pred_8 = lr.predict(X_train_8)

In [69]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_3)
auc_3 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_4)
auc_4 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_5)
auc_5 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_6)
auc_6 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_7)
auc_7 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_8)
auc_8 = metrics.auc(fpr, tpr)

In [70]:
print(auc_3)
print(auc_4)
print(auc_5)
print(auc_6)
print(auc_7)
print(auc_8)

0.5
0.5
0.49997500249975
0.501002004008016
0.500238002251879
0.49997500249975


In [71]:
# 加工した項目
X_train_63 = train_X.loc[:,['x0_age','x1_term','x63']].copy()
X_train_58 = train_X.loc[:,['x0_age','x1_term','x58']].copy()
X_train_59 = train_X.loc[:,['x0_age','x1_term','x59']].copy()
X_train_60 = train_X.loc[:,['x0_age','x1_term','x60']].copy()
X_train_68 = train_X.loc[:,['x0_age','x1_term','x68']].copy()
X_train_69 = train_X.loc[:,['x0_age','x1_term','x69']].copy()
X_train_62 = train_X.loc[:,['x0_age','x1_term','x62']].copy()
X_train_71 = train_X.loc[:,['x0_age','x1_term','x71']].copy()
X_train_67 = train_X.loc[:,['x0_age','x1_term','x67']].copy()
X_train_64 = train_X.loc[:,['x0_age','x1_term','x64']].copy()

In [72]:
X_test_63 = test_X.loc[:,['x0_age','x1_term','x63']].copy()
X_test_58 = test_X.loc[:,['x0_age','x1_term','x58']].copy()
X_test_59 = test_X.loc[:,['x0_age','x1_term','x59']].copy()
X_test_60 = test_X.loc[:,['x0_age','x1_term','x60']].copy()
X_test_68 = test_X.loc[:,['x0_age','x1_term','x68']].copy()
X_test_69 = test_X.loc[:,['x0_age','x1_term','x69']].copy()
X_test_62 = test_X.loc[:,['x0_age','x1_term','x62']].copy()
X_test_71 = test_X.loc[:,['x0_age','x1_term','x71']].copy()
X_test_67 = test_X.loc[:,['x0_age','x1_term','x67']].copy()
X_test_64 = test_X.loc[:,['x0_age','x1_term','x64']].copy()

In [73]:
lr.fit(X_train_63, train_y)
y_pred_63 = lr.predict(X_train_63)

lr.fit(X_train_58, train_y)
y_pred_58 = lr.predict(X_train_58)

lr.fit(X_train_59, train_y)
y_pred_59 = lr.predict(X_train_59)

lr.fit(X_train_60, train_y)
y_pred_60 = lr.predict(X_train_60)

lr.fit(X_train_68, train_y)
y_pred_68 = lr.predict(X_train_68)

lr.fit(X_train_69, train_y)
y_pred_69 = lr.predict(X_train_69)

lr.fit(X_train_62, train_y)
y_pred_62 = lr.predict(X_train_62)

lr.fit(X_train_71, train_y)
y_pred_71 = lr.predict(X_train_71)

lr.fit(X_train_67, train_y)
y_pred_67 = lr.predict(X_train_67)

lr.fit(X_train_64, train_y)
y_pred_64 = lr.predict(X_train_64)

In [74]:
fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_63)
auc_63 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_58)
auc_58 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_59)
auc_59 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_60)
auc_60 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_68)
auc_68 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_69)
auc_69 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_62)
auc_62 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_71)
auc_71 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_67)
auc_67 = metrics.auc(fpr, tpr)

fpr, tpr, thresholds = metrics.roc_curve(train_y, y_pred_64)
auc_64 = metrics.auc(fpr, tpr)

In [75]:
print(auc_63)
print(auc_58)
print(auc_59)
print(auc_60)
print(auc_68)
print(auc_69)
print(auc_62)
print(auc_71)
print(auc_67)
print(auc_64)

0.5
0.4999857183661811
0.5
0.5005318677422214
0.5
0.5002730746880202
0.5
0.5
0.49995715509854327
0.5004479015847689


In [76]:
import csv
with open('./test_result.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(y_pred)

In [77]:
with open('./test_result.csv') as f:
    print(f.read())

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [83]:
y_pred.shape

(42000,)